In [4]:
%reload_ext autoreload
%autoreload 2
if '__file__' not in globals():
    __file__, __name__ = globals()['__vsc_ipynb_file__'], '__ipynb__'
    import types, sys; sys.modules['__ipynb__'] = types.ModuleType('__ipynb__')

import sys, os
if os.path.abspath('.') not in sys.path: sys.path.append(os.path.abspath('.'))

from pathlib import Path
from glob import glob

class text_color:
    def __init__(self, *attrs): self.clrs = list(set(attrs))
    def __ror__(self, obj): return f'\33[{";".join(map(str, self.clrs))}m{str(obj)}\33[0m'
    @property
    def bg(self): return text_color( next((c+70 for c in self.clrs if 30 <= c < 38), self.clrs[0]))

bold, italic, underline, strike = [1,3,4,9]
black,red,green,yellow,blue,magenta,cyan,white = [text_color(clr) for clr in range(30,38)]

import inspect
class cout:
    def __ror__(self, obj): print(f'[{inspect.stack()[1].lineno}]'|green, obj)
    def __call__(self, *args, **kwds): print(f'[{inspect.stack()[1].lineno}]'|green, *args, **kwds)
out = cout()

from dotenv import load_dotenv
load_dotenv()

jjkim_key = os.getenv("JINA_API_JJKIM")
obj76_key = os.getenv("JINA_API_OBJECTS76")

from rich import print as pprint
from IPython.display import HTML, Markdown
from time import sleep

In [10]:
from window_utils import active_window_information

sleep(5)
active_window_information()

('Google Gemini - jjkim - Microsoft\u200b Edge',
 '"microsoft-edge", "Microsoft-edge"')

In [7]:
unicode_str = "ChatGPT and 15 more pages - Personal - Microsoft\u200b Edge"
unicode_str = str(unicode_str)
decoded_str = unicode_str.encode().decode('unicode_escape')
print(f"Original: {unicode_str}")
print(f"Decoded:  {decoded_str}")


Original: ChatGPT and 15 more pages - Personal - Microsoft​ Edge
Decoded:  ChatGPT and 15 more pages - Personal - Microsoftâ Edge


In [ ]:
import subprocess
def get_url(window_id:str):
    subprocess.run(['xdotool', 'key', '--window', window_id, 'ctrl+l','ctrl+c', "Escape"], check=True)
    url_result = subprocess.run(['xclip', '-selection', 'clipboard', '-o'], capture_output=True, text=True, check=True)
    url = url_result.stdout.strip()
    return url

time.sleep(5)
get_url(get_active_window())

In [ ]:
def wait_page_loaded(window_id:str, needle:str, timeout=2):
    for _ in range(timeout*10):
        result = subprocess.run(['xdotool', 'getwindowname', window_id], capture_output=True, text=True, check=True)
        if needle in result.stdout.strip():
            print('page loaded: ', result.stdout.strip())
            time.sleep(0.2)
            return True
        time.sleep(0.1)
    return False

def get_html_text(window_id:str):
    subprocess.run(['xdotool', 'key', '--window', window_id, 'ctrl+u'], check=True)
    # time.sleep(1)
    wait_page_loaded(window_id, "view-source:")
    subprocess.run(['xdotool', 'key', '--window', window_id, 'ctrl+a','ctrl+c','ctrl+w'], check=True)
    html_text = subprocess.run(['xclip', '-selection', 'clipboard', '-o'], capture_output=True, text=True, check=True)
    return html_text.stdout.strip()

time.sleep(5)
print('start')
get_html_text(get_active_window())

In [ ]:
import time
import subprocess

def wait_page_loaded(needle, timeout=3):
    cmds = "xdotool search --name --class microsoft-edge"

    result = subprocess.run(cmds.split(), capture_output=True, text=True, timeout=timeout)
    window_ids = [w for w in result.stdout.strip().split('\n') if w]

    for window_id in window_ids:
        print('activate: ', window_id)
        result = subprocess.run(['xdotool', 'windowactivate', window_id], capture_output=True, text=True, check=True, timeout=timeout)
        if result.returncode == 0 and not result.stderr.strip():
            print('windowactivate: ', result)

            result = subprocess.run(['xdotool', 'getwindowname', window_id], capture_output=True, text=True, check=True, timeout=timeout)
            if result.returncode == 0:
                title = result.stdout.strip()
                print('getwindowname: ', title)
                if needle.lower() in title.lower():
                    print('page loaded: ', title)
                    return True
    return False


wait_page_loaded('chatgpt')


In [ ]:
import subprocess
import re


def get_window_class(window_id):
    cmd = ["xprop", "-id", window_id, "WM_CLASS"]
    result = subprocess.run(cmd, stdout=subprocess.PIPE, text=True)
    win_class = re.search(r"WM_CLASS\(STRING\) = (.*)", result.stdout)
    return win_class.group(1) if win_class else None

def get_active_window():
    cmd = ["xprop", "-root", "_NET_ACTIVE_WINDOW"]
    result = subprocess.run(cmd, stdout=subprocess.PIPE, text=True)
    wnd_id = re.search(r"window id # (0x[\da-f]+)", result.stdout)
    if wnd_id:
        window_id = wnd_id.group(1)
        # get window title
        title_cmd = ["xprop", "-id", window_id, "WM_NAME"]
        title_result = subprocess.run(title_cmd, stdout=subprocess.PIPE, text=True)
        title = re.search(r"WM_NAME\(STRING\) = \"(.*)\"", title_result.stdout)
        # get window class
        return title.group(1) if title else None, get_window_class(window_id)
    return None, None

time.sleep(5)
title, win_class = get_active_window()
print("Title:", title)
print("Class:", win_class)
